## Setup

In [1]:
from functions import Model, get99, splitData, dropMissingValues, z_score, outliers_knn, outliers_dbscan, getNoise, getRelFeatures, drop_features, getCombinations, get_unique_list, bold, blue, red, green, getBestModel, np_to_df, df_to_np, drop99_all, z_score_individual, train_test_to_df
import pandas as pd
from sklearn.linear_model import LinearRegression
import warnings
import json
import numpy as np
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

In [2]:
# JSON file
f = open ('data/json/input.json', "r")
  # Reading from file
PARAMETERS = json.loads(f.read())

SPLIT_SIZE = PARAMETERS["SPLIT_SIZE"]

OUTLIER99 = PARAMETERS["OUTLIER99"] 
OUTLIER_ZSCORE = {"STD" : PARAMETERS["OUTLIER_ZSCORE"]["STD"]}
OUTLIER_ZSCORE_IND = PARAMETERS["OUTLIER_ZSCORE_IND"]
OUTLIER_KNN = {"K": PARAMETERS["OUTLIER_KNN"]["K"], "X": PARAMETERS["OUTLIER_KNN"]["X"]}
OUTLIER_DBSCAN = {"K": PARAMETERS["OUTLIER_DBSCAN"]["K"], "X": PARAMETERS["OUTLIER_DBSCAN"]["X"], "EPS": PARAMETERS["OUTLIER_DBSCAN"]["EPS"], "MIN_SAMPLES":  PARAMETERS["OUTLIER_DBSCAN"]["MIN_SAMPLES"]}
OUTLIER_NOISE = {"CV": PARAMETERS["OUTLIER_NOISE"]["CV"]}

FEATURE_SEL_CORR = {"THRESHOLD_CORR": PARAMETERS["FEATURE_SEL_CORR"]["THRESHOLD"]}

f.close()

#TODO FEATURE
#TODO Regressionmodels
#TODO TESTDATA --> path to csv-file


In [3]:
#Get raw Data
df = pd.read_csv("data/csv/house_data_training.csv", sep=';') 
# remove unnamed column
df = df.iloc[:, 1:]
#Transform string to datetime
df["date"] = pd.to_datetime(df["date"])

maeList = []#for model selection

In [4]:
df = dropMissingValues(df) 

## Outlier Detection

In [5]:
# Outlier Detection
outlier_dict_all = {}

if OUTLIER99 == True:
    print("outlier_99")
    outlier_list_99 = get99(df) #TODO Mean for price for trainingsdata
    df = drop99_all(df, outlier_list_99 )
    #outlier_dict_all["99_mean"] = outlier_list_99
    #outlier_dict_all["99_drop"] = outlier_list_99
    print(green('done'), "\n")

print('starting z_score')
for p in OUTLIER_ZSCORE["STD"]:
    print(p)
    outlier_list_z_score = z_score(df, p)
    outlier_dict_all["z_score_"+str(p)] = outlier_list_z_score

if OUTLIER_ZSCORE_IND == True:
    outlier_list_z_score_ind = z_score_individual(df)

print(green('done'), '\n\nstarting knn')
for k in OUTLIER_KNN["K"]:
    for o in OUTLIER_KNN["X"]:
        print(k, o)
        outlier_list_knn = outliers_knn(df, k, o, SPLIT_SIZE)
        outlier_dict_all["knn_"+str(k)+"_"+str(o)] = outlier_list_knn

print(green('done'), '\n\nstarting db_scan')
for k in OUTLIER_DBSCAN["K"]:
    for o in OUTLIER_DBSCAN["X"]:
        for e in OUTLIER_DBSCAN["EPS"]:
            for m in OUTLIER_DBSCAN["MIN_SAMPLES"]:
                print(k, o, e, m)
                outlier_list_dbscan = outliers_dbscan(df, k, o, e, m, SPLIT_SIZE)
                outlier_dict_all["db_scan"] = outlier_list_dbscan

print(green('done'), '\n\nstarting noise')
for cv in OUTLIER_NOISE["CV"]:
        print(cv)
        outlier_list_noise = getNoise(df, cv)
        outlier_dict_all["noise_"+str(cv)] = outlier_list_noise
print(green('done'))

outlier_99
done 

starting z_score
10.1
done 

starting knn
3 200
done 

starting db_scan
3 200 0.42 5
done 

starting noise
5
done


In [6]:
outlier_lists_all = list(outlier_dict_all.values())
outlier_lists_keys = list(outlier_dict_all.keys())
combination_list = []
for i, combo in enumerate(getCombinations(outlier_lists_keys), 1):
    if not i == 1: #emtpy tuple
        combination_list.append(combo)
        # print('combo #{}: {}'.format(i, combo))
print("...")
for c in combination_list[10:]:
    print(bold("Combination: "), blue(c),"\n",)
print(bold(len(combination_list)), " total Combinations")

...
Combination:  ('z_score_10.1', 'knn_3_200', 'db_scan') 

Combination:  ('z_score_10.1', 'knn_3_200', 'noise_5') 

Combination:  ('z_score_10.1', 'db_scan', 'noise_5') 

Combination:  ('knn_3_200', 'db_scan', 'noise_5') 

Combination:  ('z_score_10.1', 'knn_3_200', 'db_scan', 'noise_5') 

15  total Combinations


In [7]:
model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        
        # if "99_mean" in c:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "both", outlier_list_99)
        # else:
        #     X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        #df_price_rem = df.drop(columns=["price"])
        # print(df_price_rem[:2])
        #print(df[:1])
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop") #TODO drop entfernen

        #TODO Regressionmodels
        model = LinearRegression()
        obj = Model(model, (X_train, X_test, y_train, y_test), c, df, df.columns.to_list())
        model_obj_list.append(obj)

    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 53 rows

1: ('knn_3_200',)
dropped 200 / 200 rows

2: ('db_scan',)
dropped 2583 / 2583 rows

3: ('noise_5',)
dropped 35 / 51 rows

4: ('z_score_10.1', 'knn_3_200')
dropped 239 / 239 rows

5: ('z_score_10.1', 'db_scan')
dropped 2615 / 2615 rows

6: ('z_score_10.1', 'noise_5')
dropped 76 / 76 rows

7: ('knn_3_200', 'db_scan')
dropped 2583 / 2583 rows

8: ('knn_3_200', 'noise_5')
dropped 235 / 235 rows

9: ('db_scan', 'noise_5')
dropped 2612 / 2612 rows

10: ('z_score_10.1', 'knn_3_200', 'db_scan')
dropped 2615 / 2615 rows

11: ('z_score_10.1', 'knn_3_200', 'noise_5')
dropped 274 / 274 rows

12: ('z_score_10.1', 'db_scan', 'noise_5')
dropped 2644 / 2644 rows

13: ('knn_3_200', 'db_scan', 'noise_5')
dropped 2612 / 2612 rows

14: ('z_score_10.1', 'knn_3_200', 'db_scan', 'noise_5')
dropped 2644 / 2644 rows


In [8]:
#model_obj_list = []
for i, c in enumerate(combination_list):
    print(f'\n{bold(i)}: {blue(c)}')
    outlier_list = get_unique_list(outlier_dict_all, c)
    try: 
        X_train, X_test, y_train, y_test = splitData(df, SPLIT_SIZE, outlier_list, "drop")
        df_lasso = train_test_to_df(X_train, X_test, y_train, y_test, df.columns.to_list())
        #print(df_lasso)
        #print(df_lasso[:1])
        # X = np.append(X_train, X_test, axis=0)
        # y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
        # X_y = np.append(X, y, axis=1)
        # features = df.columns.to_list()
        # df_adsa = pd.DataFrame(X_y, columns=features)

        X_l, y_l = df_lasso.drop(columns=["price", "date", "id"]), df_lasso["price"]
        scaler = StandardScaler().fit(X_l)
        X_train_l, X_test_l, y_train_l, y_test_l = train_test_split(X_l, y_l, test_size = SPLIT_SIZE, random_state=1) #TODO 
        #rint(X_train_l)
        X_train_l = scaler.transform(X_train_l)
        X_test_l = scaler.transform(X_test_l)
        reg = Lasso(alpha=1)
        obj_l = Model(reg, (X_train_l, X_test_l, y_train_l, y_test_l), c, df_lasso, X_l.columns.to_list())
        model_obj_list.append(obj_l)
        ''''''
    except Exception as e:
        print(bold("Error"), red(e))


0: ('z_score_10.1',)
dropped 41 / 41 rows

1: ('knn_3_200',)
dropped 200 / 200 rows

2: ('db_scan',)
dropped 2583 / 2583 rows

3: ('noise_5',)
dropped 35 / 35 rows

4: ('z_score_10.1', 'knn_3_200')
dropped 239 / 239 rows

5: ('z_score_10.1', 'db_scan')
dropped 2615 / 2615 rows

6: ('z_score_10.1', 'noise_5')
dropped 76 / 76 rows

7: ('knn_3_200', 'db_scan')
dropped 2583 / 2583 rows

8: ('knn_3_200', 'noise_5')
dropped 235 / 235 rows

9: ('db_scan', 'noise_5')
dropped 2612 / 2612 rows

10: ('z_score_10.1', 'knn_3_200', 'db_scan')
dropped 2615 / 2615 rows

11: ('z_score_10.1', 'knn_3_200', 'noise_5')
dropped 274 / 274 rows

12: ('z_score_10.1', 'db_scan', 'noise_5')
dropped 2644 / 2644 rows

13: ('knn_3_200', 'db_scan', 'noise_5')
dropped 2612 / 2612 rows

14: ('z_score_10.1', 'knn_3_200', 'db_scan', 'noise_5')
dropped 2644 / 2644 rows


In [17]:
df_lasso.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,dis_super,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,ahf1,ahf2,ahf3,price
0,3.438501e+09,1.418602e+18,3.0,1.00,970.0,6828.0,1.0,0.0,640.05,0.0,...,0.0,98106.0,47.5476,-122.360,1160.0,11666.0,39.376922,114.928298,154.305220,315000.0
1,2.598010e+08,1.425859e+18,4.0,2.00,1610.0,8000.0,1.0,0.0,739.03,0.0,...,0.0,98008.0,47.6301,-122.118,1560.0,7896.0,82.463087,97.001643,179.464731,526000.0
2,3.832060e+09,1.426464e+18,4.0,2.50,2200.0,5893.0,2.0,0.0,501.91,0.0,...,0.0,98042.0,47.3333,-122.055,2200.0,5757.0,58.893575,119.451791,178.345367,280000.0
3,1.722069e+09,1.419811e+18,3.0,2.50,3100.0,100188.0,1.0,0.0,909.06,0.0,...,0.0,98038.0,47.3928,-122.066,2430.0,104979.0,66.186721,109.383269,175.569991,540000.0
4,8.722101e+09,1.430266e+18,3.0,2.25,2610.0,3357.0,2.0,0.0,1043.84,0.0,...,0.0,98112.0,47.6380,-122.306,1940.0,3357.0,39.024317,93.887967,132.912284,1049000.0


In [10]:
# def getDF(self):
#     X_train, X_test, y_train, y_test = self.get_data()
#     print(len(y_test))
#     #print(y_train.shape, y_test.reshape(len(y_test), 1).shape)
#     X = np.append(X_train, X_test, axis=0)
#     y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
#     X_y = np.append(X, y, axis=1)
#     features = df.columns.to_list()
#     features.remove("price")
#     df = pd.DataFrame(X_y, columns=features)
#     return df

In [11]:
#Summary
df_summary = pd.DataFrame(columns=['model', 'combo', 'mae', 'score'])
for o in model_obj_list:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    row = {'model': o.get_model(), 'combo': comb, 'mae': mae, 'score': score}
    df_summary = df_summary.append(row, ignore_index=True)

df_summary.sort_values("mae")

,model,combo,mae,score
22,Lasso(alpha=1),"(knn_3_200, db_scan)",103538.82,0.69365
17,Lasso(alpha=1),"(db_scan,)",103538.82,0.69365
24,Lasso(alpha=1),"(db_scan, noise_5)",104526.17,0.685343
28,Lasso(alpha=1),"(knn_3_200, db_scan, noise_5)",104526.17,0.685343
25,Lasso(alpha=1),"(z_score_10.1, knn_3_200, db_scan)",105265.91,0.68466
20,Lasso(alpha=1),"(z_score_10.1, db_scan)",105265.91,0.68466
29,Lasso(alpha=1),"(z_score_10.1, knn_3_200, db_scan, noise_5)",106125.63,0.690928
27,Lasso(alpha=1),"(z_score_10.1, db_scan, noise_5)",106125.63,0.690928
19,Lasso(alpha=1),"(z_score_10.1, knn_3_200)",116016.8,0.702118
23,Lasso(alpha=1),"(knn_3_200, noise_5)",116945.08,0.686948


In [15]:
number = 1
print(model_obj_list[number])
lassotest = model_obj_list[number]
# lassotest = model_obj_list[number]
# print(lassotest.get_model())
print(lassotest.get_data()[0].shape, lassotest.get_data()[1].shape , lassotest.get_data()[2].shape, lassotest.get_data()[3].shape )
# X_train, X_test, y_train, y_test = lassotest.get_data()
# print(X_train.shape)
# lassotest.get_model().fit(X_train, y_train)
# y_pred = lassotest.get_model().predict(X_test)
# print("mae", mean_absolute_error(y_test, y_pred))
# print(lassotest.get_df())
lassotest.describe()

Model: LinearRegression() Combination: ('knn_3_200',) Features: 25
(10723, 24) (3642, 24) (10723, 1) (3642,)


AttributeError: 'Model' object has no attribute 'describe'

In [13]:
#row
mae_best = df_summary[df_summary.mae==df_summary.mae.min()]
#print(mae_best)
#score_best = df_summary[df_summary.score==df_summary.score.max()]
best_model_obj = getBestModel(model_obj_list, df_summary, mae_best.index[0])
print(best_model_obj)
#rint(f'{bold("Lowest mae:")} {mae_best}')
#model_obj_list

Model: Lasso(alpha=1) Combination: ('db_scan',) Features: 22


## Feature Selection

In [14]:
model_obj_list_best = [best_model_obj]
X_train, X_test, y_train, y_test = best_model_obj.get_data()

df_feature_reduction = train_test_to_df(X_train, X_test, y_train, y_test, best_model_obj.get_features())
print(df_feature_reduction.columns)
#print(df_feature_reduction)

for threshold in FEATURE_SEL_CORR["THRESHOLD_CORR"]:
    #print(threshold, type(threshold))
    try: 
        rel_features = getRelFeatures(df_feature_reduction, threshold)
        print(len(rel_features), rel_features)
        df_feature_reduction = df_feature_reduction[rel_features]
        X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
        best_model_obj_new = Model(best_model_obj.get_model(), (X_train, X_test, y_train, y_test), best_model_obj.get_comb() , df_feature_reduction, df.columns.to_list())
        model_obj_list_best.append(best_model_obj_new)

    except Exception as e:
        print(bold("Error"), red(e))
print(model_obj_list_best)

ValueError: list.remove(x): x not in list

In [ ]:
#Summary
df_summary_best = pd.DataFrame(columns=['model', 'combo', 'mae', 'score', 'features'])
for o in model_obj_list_best:
    #print(f'{o.get_type()}, {o.get_comb()}')
    o.train()
    mae, score = o.summary()
    comb = o.get_comb()
    features = o.get_df().columns.to_list()
    if "price" in features:
        features.remove("price")
    row = {'model': o.get_model(),'combo': comb, 'mae': mae, 'score': score, 'features': features}
    df_summary_best = df_summary_best.append(row, ignore_index=True)

df_summary_best.sort_values("mae")

,model,combo,mae,score,features
1,LinearRegression(),"(z_score_10.1, db_scan, noise_5)",0.51,0.347769,"[bathrooms, grade, bedrooms, long, condition, ..."
0,LinearRegression(),"(z_score_10.1, db_scan, noise_5)",161768.15,0.449157,"[id, date, bedrooms, bathrooms, sqft_living, s..."


In [ ]:
#row
mae_best = df_summary_best[df_summary_best.mae==df_summary_best.mae.min()]
#score_best = df_summary_best[df_summary_best.score==df_summary_best.score.max()]
best_model_obj = getBestModel(model_obj_list_best, df_summary_best, mae_best.index[0])
print(best_model_obj)
print(f'{bold("Lowest mae:")} {mae_best}')


Model: LinearRegression() Combination: ('z_score_10.1', 'db_scan', 'noise_5') Features: 24
Lowest mae:                 model                             combo        mae     score  \
0  LinearRegression()  (z_score_10.1, db_scan, noise_5)  161768.15  0.449157   

                                            features  
0  [id, date, bedrooms, bathrooms, sqft_living, s...  


In [ ]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,dis_super,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,ahf1,ahf2,ahf3
0,7212650950,2014-07-08,336000.0,4,2.50,2530,8169,2.0,0.0,562.16,...,1993.0,0,98003,47.2634,-122.312,2220,8013,24.658459,108.183999,132.842458
1,3616600250,2014-05-27,1600000.0,3,3.25,3790,19000,2.0,0.0,1462.74,...,1985.0,0,98177,47.7240,-122.373,2740,18628,18.682061,83.213016,101.895077
2,1826049225,2015-04-17,460000.0,4,1.75,1870,8663,1.0,0.0,599.53,...,1949.0,0,98133,47.7366,-122.350,1560,7800,75.078050,101.082827,176.160876
3,1337800830,2015-01-07,998500.0,3,1.75,2140,4800,2.0,0.0,970.84,...,1905.0,0,98112,47.6311,-122.312,2440,4800,49.367640,97.749706,147.117347
4,3062600050,2014-07-14,745000.0,3,2.75,3010,12432,1.0,0.0,1157.55,...,1970.0,0,98052,47.6392,-122.108,2500,12432,43.343635,98.282451,141.626086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14992,5700000905,2014-08-16,739000.0,5,2.50,2840,5000,1.0,0.0,694.64,...,1913.0,0,98144,47.5817,-122.291,2200,5000,24.130536,112.985690,137.116226
14993,2623069106,2015-02-19,710000.0,6,3.50,3830,68825,2.0,0.0,1156.61,...,1995.0,0,98027,47.4574,-122.003,2410,68825,83.830652,90.383630,174.214282
14994,1530900290,2014-10-07,475000.0,3,2.50,2280,3710,1.0,0.0,1352.78,...,1990.0,0,98072,47.7350,-122.159,2030,3710,13.268193,117.766765,131.034957
14995,4310701600,2014-11-13,340000.0,3,2.50,1240,1115,3.0,0.0,1245.41,...,2003.0,0,98103,47.6985,-122.340,1410,1355,48.352194,102.297062,150.649256


In [ ]:
number = 15
print(model_obj_list[number])
lassotest = model_obj_list[number]
X_train, X_test, y_train, y_test =  lassotest.get_data()
X = np.append(X_train, X_test, axis=0)
print(type(y_test))
y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

X_y = np.append(X, y, axis=1)
features = df.columns.to_list()
df_adsa = pd.DataFrame(X_y, columns=features)

df_adsa

Model: Lasso(alpha=1) Combination: ('z_score_10.1',) Features: 24
<class 'pandas.core.series.Series'>


AttributeError: 'Series' object has no attribute 'reshape'

In [ ]:
def train_test_to_df(X_train, X_test, y_train, y_test, columns):
    X = np.append(X_train, X_test, axis=0)
    columns.remove("price")
    columns.append("price")
    #print(X_train.shape)
    print("X", X.shape)
    try:
        y = np.append(y_train, y_test.reshape(len(y_test), 1), axis=0)

    except:
        y = np.append(y_train, y_test, axis=0)
        y = y.reshape(len(y), 1)
        
    print("y", y.shape)
    X_y = np.append(X, y, axis=1)
    print("X_y", X_y.shape)
    #features = df.columns.to_list()
    df = pd.DataFrame(X_y, columns=columns)
    return df

In [ ]:
# def train_test_to_df(X_train, X_test, y_train, y_test, columns):
#     X_train = pd.DataFrame(X_train, columns = df.columns.to_list().remove("price"))
#     X_test = pd.DataFrame(X_test, columns = df.columns.to_list().remove("price"))
#     y_train = pd.DataFrame(y_train, columns  = ["price"])
#     y_test = pd.DataFrame(y_test, columns = ["price"])

    
#     return df

In [ ]:
#print(df.columns.to_list())
X_train, X_test, y_train, y_test =  lassotest.get_data()

columnsList = lassotest.get_features()
#columnsList.remove("price")

#columnsList.append("price")
print(X_train.shape, X_test.shape,y_train.shape, y_test.shape)
train_test_to_df(X_train, X_test, y_train, y_test, columnsList)

(10893, 22) (3631, 22) (10893,) (3631,)
X (14524, 22)
y (14524, 1)
X_y (14524, 23)


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,dis_super,view,condition,grade,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,ahf1,ahf2,ahf3,price
0,-0.796337,-0.782488,-0.353438,2.802038,-0.083264,-0.986579,-0.302949,-0.628109,-0.553934,-0.566864,...,0.568401,0.821662,-1.260859,-0.967035,-0.424855,-0.286571,2.308098,0.918924,-0.269110,2.0
1,0.831897,0.712641,-0.204478,0.944373,-0.083264,0.363863,-0.302949,-0.628109,0.297244,1.202671,...,-0.583495,-0.500578,3.205677,0.920578,-0.237132,0.046800,-1.034629,-0.482337,0.016332,4.0
2,-0.796337,-0.599411,-0.304919,0.944373,-0.083264,1.046153,-0.302949,2.456995,-0.553934,-0.446488,...,0.494085,0.955443,-0.973160,-0.734713,-0.331161,-1.089164,-1.136907,-1.513629,0.330951,4.0
3,0.506250,-0.263769,-0.188589,0.944373,-0.083264,-0.110975,-0.302949,0.914443,0.297244,0.047056,...,-0.639232,-1.168069,0.436568,0.121973,-0.189690,1.118275,2.089247,2.019858,-0.553564,3.0
4,-0.470690,-0.538385,-0.198520,-0.913292,-0.083264,-2.045078,-0.302949,-0.628109,-1.405113,-1.120596,...,0.717033,1.031890,-0.462493,-0.821834,-0.203982,-0.262610,0.809294,0.182389,-0.187838,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14519,-1.447630,-0.273940,-0.281060,0.944373,-0.083264,-2.148506,-0.302949,-0.628109,0.297244,0.035018,...,0.475506,0.506674,-0.764577,0.092933,-0.322771,-1.717783,-0.973267,-1.973060,0.800977,4.0
14520,1.808837,1.526317,-0.246651,0.944373,-0.083264,-0.033404,2.333383,-0.628109,1.148423,1.623988,...,0.717033,0.863424,-0.376183,0.165533,-0.239248,0.797221,0.943321,1.164067,2.093208,4.0
14521,-0.145043,-0.223086,-0.240306,-0.913292,-0.083264,-0.994391,-0.302949,-0.628109,0.297244,0.095207,...,-0.434864,1.102674,1.321245,-0.458831,-0.304210,-1.224718,-1.470016,-1.798815,0.066816,2.0
14522,-0.470690,-0.192573,-0.195605,-0.913292,-0.083264,-0.455227,-0.302949,-0.628109,-0.553934,0.131319,...,-0.787864,1.251319,-0.124446,-0.821834,-0.197152,1.670065,-1.395888,0.735010,-0.512927,4.0


In [ ]:
rel = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'dis_super', 'view', 'condition',
       'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'ahf1', 'ahf2',
       'ahf3']
from functions import train_test_split
from sklearn.preprocessing import StandardScaler
X, y = df_adsa.drop(columns=["price", "date", "id"]), df_adsa["price"]
scaler = StandardScaler().fit(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = SPLIT_SIZE, random_state=1) 

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error

reg = Lasso(alpha=1)
reg.fit(X_train, y_train)
print(reg.score(X_test, y_test))
y_pred = reg.predict(X_test)
print("dsds", mean_absolute_error(y_test, y_pred))

-0.00015689062718582392
dsds 0.7348257015548852


## Regressionmodels

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
poly.fit_transform(best_model_obj_new.get_df())




NameError: name 'best_model_obj_new' is not defined

## Evaluatuion

## Testdata

In [ ]:
# combination_list = []
# df_feature_reduction = best_model_obj.get_df()

# for i, combo in enumerate(getCombinations(df_feature_reduction.columns.to_list()), 1):
#     if not i == 1: #emtpy tuple
#         combination_list.append(combo)
#         # print('combo #{}: {}'.format(i, combo))
# print("...")
# for c in combination_list[10:]:
#     print(bold("Combination: "), blue(c),"\n",)
# print(bold(len(combination_list)), " total Combinations")

In [ ]:
# model_obj_list_best = []
# df_feature_reduction = best_model_obj.get_df()
# for i, c in enumerate(combination_list):
#     print(f'\n{bold(i)}: {blue(c)}')
    
#     try: 
#         X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#         best_model_obj_copy = best_model_obj.copy()
#         best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

#         model_obj_list_best.append(obj)

#     except Exception as e:
#         print(bold("Error"), red(e))

In [ ]:
# df_feature_reduction = best_model_obj.get_df()
# getCombinations(df_feature_reduction.columns.to_list())

# for feature in list(df_feature_reduction.columns.to_list()):
#     df_feature_reduction.drop(columns=feature)
#     X_train, X_test, y_train, y_test = splitData(df_feature_reduction, SPLIT_SIZE)
#     best_model_obj_copy = best_model_obj.copy()
#     best_model_obj_copy.set_data(X_train, X_test, y_train, y_test)

In [ ]:
# #Wrapper approaches
# #Backward elimination using Recursive feature elimination¶
# from sklearn.svm import SVC
# from sklearn import feature_selection
# import pandas as pd
# import numpy as np


# x, y = df.drop(columns=["price"]), df["price"]
# # x_y= np.concatenate((x.reshape(150,4),y.reshape(150,1)),1)
# x_y= np.concatenate((x,y),1)
# #print(x_y)
# #print(iris.target_names)

# #Create column list
# df_columns=df.columns
# # df_columns.append("Label")
# # #Create PANDAS data frame
# # df = pd.DataFrame(x_y,columns=df_columns)
# #Map label index to label name
# df['price']=df['price'].map(lambda x: iris.target_names[int(x)])
# print("\nOriginal Iris Data Set:")
# print(df)

# #Create the RFE object and rank features
# num_features=2
# svc = SVC(kernel="linear", C=1)
# rfe = feature_selection.RFE(estimator=svc, n_features_to_select=num_features, step=1)
# rfe.fit(x, y)
# #print("Selected features will have a ranking=1 and support=TRUE")
# #print(iris.feature_names," ",rfe.ranking_," ",rfe.support_)
# #print(x.shape)

# #extend column-mask by one column for the label (always true)
# column_mask=np.append(rfe.support_,True)
# #use list column_mask to mask df-columns list
# reduced_iris_features = [df_columns[i] for i in range(len(df_columns)) if column_mask[i]]
# print("Reduced_iris_features: ",reduced_iris_features)
# # use reduced_iris_features to reduce the data frame
# reduced_df=df[reduced_iris_features]

# print("\nIris Data Set reduced to ",num_features," features: \n",reduced_df)

In [ ]:
#TODO sqft_living wenn price auffällig niedrig obwohl m² hoch clsuter
#TODO PCA macht warscheinlich wenig sinn